## ARWU-web-crawler for Global Ranking of Academic Subjects

target link:  
- [Global Ranking of Academic Subjects link](https://www.shanghairanking.com/rankings/gras/2021)
  
- [Mathematics subject link](https://www.shanghairanking.com/rankings/gras/2021/RS0101)

In [2]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from selenium.webdriver.common.keys import Keys

print("installed finish")

installed finish


In [10]:
from selenium.webdriver.support.ui import Select
# test for open driver
option = Options()
option.add_argument("--disable-notifications")
chrome = webdriver.Chrome('./chromedriver', options=option)

url = "http://www.shanghairanking.com/rankings/gras/2021"
chrome.get(url)
soup = BeautifulSoup(chrome.page_source, 'html.parser')
chrome.close()
print("finish")

finish


### Area Name

In [11]:
## get 51 subject name
url = "http://www.shanghairanking.com/rankings/gras/2021"
response = requests.get(url)
soup = BeautifulSoup(response.text, "lxml")

area_list = []
areas = soup.find_all(class_="subject-title")

for area in areas:
    area_list.append(area.text)
    print(area.text.lstrip().rstrip())
print("Area numbers:", len(area_list))

Natural Sciences
Engineering
Life Sciences
Medical Sciences
Social Sciences
Area numbers: 5


### Subject Name

In [20]:
subject_dict = {}
subjects = soup.find_all(class_="subj-link")

for subject in subjects:
    link = subject["href"].split("/")[-1]
    subject_dict[subject.text] = link
    
count = 0
for key, value in subject_dict.items():
    print("%-15s : %6s" % (key, value))
    if(count > 5):
        break
    else:
        count += 1

Mathematics     : RS0101
Physics         : RS0102
Chemistry       : RS0103
Earth Sciences  : RS0104
Geography       : RS0105
Ecology         : RS0106
Oceanography    : RS0107


In [25]:
# find limited page number
def get_final_page_number(sub_num):
    subject_url = "http://www.shanghairanking.com/rankings/gras/2021/" + sub_num
    chrome = webdriver.Chrome('./chromedriver', options=option)
    chrome.get(subject_url)

    soup = BeautifulSoup(chrome.page_source, 'lxml')
    flag = False
    tags_a = soup.find_all("a")
    limited_page = 0
    for tag in tags_a:
        if(tag.text == "•••" and flag == False):
            flag = True
        elif(flag == True):
            try:
                limited_page = int(tag.text)
            except:
                print("ERROR: Error with convert page number to integer.")
            break
    print("limit page:", limited_page)
    chrome.close()
    return limited_page

In [31]:
# get one page's infomation
def get_page_info(soup, tag):
    print(f"{tag} ", end="")
    items = soup.find_all("tr")
    flag = False
    lists = [[], [], [], []]

    for row in items:
        if(flag == False):
            flag = True
            continue
        single_list = []
        # score and ranking
        for i in [0,3,4]:
            try:
                text = row.contents[i].text.lstrip().rstrip()
                single_list.append(text)
            except:
                print("ERROR: score ranking error.")
                pass
        # institution name
        try:
            text = row.find(class_="univ-name").text.lstrip().rstrip()
            single_list.append(text)
        except:
            try:
                text = row.find(class_="univ-name-normal").text.lstrip().rstrip()
                single_list.append(text)
            except:
                print("ERROR: university name error.")
                pass
        
        if(len(single_list) == 4):
            for i in range(4):
                lists[i].append(single_list[i])
        else:
            print(single_list, " row error.")

    return lists

In [29]:
# fill in data list with result
def list_append_all(result, list1, list2, list3, list4):
    list1 += result[0]     # ranking
    list2 += result[1]     # total score
    list3[0] += result[2]  # item score
    list4 += result[3]     # institution name

### Crawl a subject 

Crawl a subject ranking with 5 scores. Get 5 scores for each university in the same page, and then switch to next page to get another universities' different scores.

sample output :   
\[Page:  1\] Q1 CNCI IC TOP AWARD  
\[Page:  2\] Q1 CNCI IC TOP AWARD  
...

In [45]:
def crawl_subject(link):
    subject_url = "http://www.shanghairanking.com/rankings/gras/2021/" + link
    chrome = webdriver.Chrome('./chromedriver', options=option)
    chrome.get(subject_url)

    count = 0
    limited_page = get_final_page_number(link)
    univ_name = []
    total_score = []
    item_score = [[] for i in range(5)]
    ranking = []

    while(count < limited_page):
        time.sleep(1)
        print("[Page: %2d] " % (count + 1), end="")
        # 5 score for a page
        for index in range(5):
            btns = chrome.find_elements_by_class_name("head-bg", )
            btns[-1].click()

            links = chrome.find_elements_by_tag_name("li")
            # for 5 item score
            btn_name_list = ["Q1", "CNCI", "IC", "TOP", "AWARD"]
            for link in links:
                if(link.text == btn_name_list[index]):
                    link.click()
                    soup = BeautifulSoup(chrome.page_source, 'lxml')
                    result = get_page_info(soup, btn_name_list[index])
                    # start: construct all lists' info
                    if(index == 0):
                        list_append_all(result, ranking, total_score, item_score, univ_name)
                    # only fill in item score
                    else:
                        item_score[index] += result[2]
                    time.sleep(2)
                    break  # to click option button
        # click next page
        btns_next_page = chrome.find_elements_by_class_name("ant-pagination-item-link", )
        btns_next_page[-1].click()
        count += 1
        # scroll to top
        js="var action=document.documentElement.scrollTop=0"
        chrome.execute_script(js)
        time.sleep(1)
        print()

    chrome.close()
    print("- finish -")
    return univ_name, total_score, item_score, ranking

### Main Function

For each subject, crawl its ranking data and output to the file.

In [48]:
import pandas as pd
for subject, link in subject_dict.items():
    print(subject, link)
    univ_name, total_score, item_score, ranking = crawl_subject(link)
    
    # write file
    df = pd.DataFrame()
    df["Year"] = ["2021" for i in range(len(ranking))]
    # df["Field"] = ["" for i in range(len(ranking))]
    # df["Subject"] = [subject for i in range(len(ranking))]
    df["World Rank"] = ranking
    df["Institution"] = univ_name
    df["Total Score"] = total_score
    
    name_list = ["Q1", "CNCI", "IC", "TOP", "AWARD"]
    for index in range(len(name_list)):
        df[name_list[index]] = item_score[index]
        
    df.to_csv(f"ARWU-subject/{subject}.csv", index=False)
    print("[ write finish ]")

Mathematics RS0101
limit page: 17
[Page:  1] Q1 CNCI IC TOP AWARD 
[Page:  2] Q1 CNCI IC TOP AWARD 
[Page:  3] Q1 CNCI IC TOP AWARD 
[Page:  4] Q1 CNCI IC TOP AWARD 
[Page:  5] Q1 CNCI IC TOP AWARD 
[Page:  6] Q1 CNCI IC TOP AWARD 
[Page:  7] Q1 CNCI IC TOP AWARD 
[Page:  8] Q1 CNCI IC TOP AWARD 
[Page:  9] Q1 CNCI IC TOP AWARD 
[Page: 10] Q1 CNCI IC TOP AWARD 
[Page: 11] Q1 CNCI IC TOP AWARD 
[Page: 12] Q1 CNCI IC TOP AWARD 
[Page: 13] Q1 CNCI IC TOP AWARD 
[Page: 14] Q1 CNCI IC TOP AWARD 
[Page: 15] Q1 CNCI IC TOP AWARD 
[Page: 16] Q1 CNCI IC TOP AWARD 
[Page: 17] Q1 CNCI IC TOP AWARD 
- finish -
[ write finish ]
